In [130]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [131]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [132]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential

# **Preprocessing**

**Train**

In [133]:
train_datagen = ImageDataGenerator(
    rescale=1./255,          # normalize pixel values to 0–1
    shear_range=0.2,         # random geometric distortions
    zoom_range=0.2,          # zoom in/out randomly
    horizontal_flip=True     # mirror flip images
)

In [134]:
training_set = train_datagen.flow_from_directory('D:\Project\FaceDetection\Train', target_size=(128, 128), batch_size=32, class_mode='binary',seed = 42, classes=['WithMask','WithoutMask'])

Found 10000 images belonging to 2 classes.


**Test**

In [135]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [136]:
test_set = test_datagen.flow_from_directory('D:\Project\FaceDetection\Test', target_size=(128, 128), batch_size=32, class_mode='binary',seed = 42, classes=['WithMask','WithoutMask'])

Found 992 images belonging to 2 classes.


# **Model**

In [137]:
cnn = Sequential()

**1st Convolution**

In [138]:
# 1️⃣ First Convolution + Pooling
cnn.add(tf.keras.layers.Conv2D(
    filters=32, kernel_size=3, padding="same",
    activation='relu', input_shape=[128,128,3]
))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# 2️⃣ Second Convolution + Pooling
cnn.add(tf.keras.layers.Conv2D(
    filters=64, kernel_size=3, padding="same",
    activation='relu', use_bias=False
))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# 3️⃣ Third Convolution + Pooling
cnn.add(tf.keras.layers.Conv2D(
    filters=128, kernel_size=3, padding="same",
    activation='relu', use_bias=False
))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# 4️⃣ Flattening
cnn.add(tf.keras.layers.Flatten())

# 5️⃣ Fully Connected Layer
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dropout(0.5))

# 6️⃣ Output Layer (Binary Classification)
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

**pooling**

**2nd covolution layer**

**3rd**

**flattening**

**connections**

**outputlayer**

# **Training**

In [139]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
cnn.fit(x=training_set, validation_data=test_set, epochs=20)


Epoch 1/20
313/313 [==============================] - 308s 980ms/step - loss: 0.1420 - accuracy: 0.9460 - val_loss: 0.0652 - val_accuracy: 0.9748
Epoch 2/20
 16/313 [>.............................] - ETA: 46s - loss: 0.0631 - accuracy: 0.9805

# single prediction single photo

In [ ]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('D:\\Project\\FaceDetection\Validation\\WithMask\\145.png', target_size = (128,128))
test_image = image.img_to_array(test_image)
test_image=test_image/255
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)

1/1 [==============================] - 0s 93ms/step


In [ ]:
if result[0][0] == 1:
  prediction = "WithMask"
else:
  prediction = "WithoutMask"

In [ ]:
print(prediction)

WithoutMask


In [ ]:
import numpy as np

predictions = cnn.predict(test_set)
predicted_labels = (predictions > 0.5).astype(int)

# Count predicted classes
unique, counts = np.unique(predicted_labels, return_counts=True)
print(dict(zip(unique, counts)))

31/31 [==============================] - 14s 454ms/step
{0: 466, 1: 526}
